In [ ]:
# Import necessary libraries
import os
import cv2
import gc
import shutil
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob
import seaborn as sns
import tensorflow as tf
from keras import layers
from sklearn import metrics
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input,Dropout,BatchNormalization

In [ ]:
# Create a new directory for working with the data

if not os.path.exists('E:/python/deep_learning_project/dataset1/datasets'):
    os.makedirs('E:/python/deep_learning_project/dataset1/datasets') 

In [ ]:
# Create a new directory for working with the data

if not os.path.exists('E:/python/deep_learning_project/dataset1/datasets/training'):
    os.makedirs('E:/python/deep_learning_project/dataset1/datasets/training') 

if not os.path.exists('E:/python/deep_learning_project/dataset1/datasets/testing'):
    os.makedirs('E:/python/deep_learning_project/dataset1/datasets/testing') 

if not os.path.exists('E:/python/deep_learning_project/dataset1/datasets/validation'):
    os.makedirs('E:/python/deep_learning_project/dataset1/datasets/validation') 

In [ ]:
if not os.path.exists("E:/python/deep_learning_project/dataset1/datasets/training/normal"):
    os.makedirs("E:/python/deep_learning_project/dataset1/datasets/training/normal")
if not os.path.exists("E:/python/deep_learning_project/dataset1/datasets/training/sick"):
    os.makedirs("E:/python/deep_learning_project/dataset1/datasets/training/sick")
if not os.path.exists("E:/python/deep_learning_project/dataset1/datasets/testing/normal"):
    os.makedirs("E:/python/deep_learning_project/dataset1/datasets/testing/normal")
if not os.path.exists("E:/python/deep_learning_project/dataset1/datasets/testing/sick"):
    os.makedirs("E:/python/deep_learning_project/dataset1/datasets/testing/sick")
if not os.path.exists("E:/python/deep_learning_project/dataset1/datasets/validation/normal"):
    os.makedirs("E:/python/deep_learning_project/dataset1/datasets/validation/normal")
if not os.path.exists("E:/python/deep_learning_project/dataset1/datasets/validation/sick"):
    os.makedirs("E:/python/deep_learning_project/dataset1/datasets/validation/sick")

In [ ]:
main_path  = 'E:/python/deep_learning_project/dataset1/known_images'
images = os.listdir(main_path)

normal_images = []
sick_images = []

# seperate normal images from sick images
for image in images:
    if image.startswith('normal'):
        normal_images.append(image)
    else:
        sick_images.append(image)

In [ ]:
# perform train test and validation split for normal images
def populate_normal_images(normal_images, path):
    for i,img in enumerate(normal_images):
        if i < 250:
            shutil.copy(os.path.join(path, img),
                      os.path.join("E:/python/deep_learning_project/dataset1/datasets/training/normal", img))
        elif i > 250 and i < (250+50):
            shutil.copy(os.path.join(path, img),
                      os.path.join("E:/python/deep_learning_project/dataset1/datasets/testing/normal", img))
        else:
              if i > 364:
                return
              shutil.copy(os.path.join(path, img),
                          os.path.join("E:/python/deep_learning_project/dataset1/datasets/validation/normal", img))
    return
    
populate_normal_images(normal_images,main_path )

In [ ]:
# perform train test and validation split for  sick images
def populate_sick_images(sick_images,path):
    for i,img in enumerate(sick_images):
        if i < 250:
            shutil.copy(os.path.join(path, img),
                      os.path.join("E:/python/deep_learning_project/dataset1/datasets/training/sick", img))
        elif i > 250 and i < (250+50):
            shutil.copy(os.path.join(path, img),
                      os.path.join("E:/python/deep_learning_project/dataset1/datasets/testing/sick", img))
        else:
              if i > (351):
                return
              shutil.copy(os.path.join(path, img),
                          os.path.join("E:/python/deep_learning_project/dataset1/datasets/validation/sick", img))
    return

populate_sick_images(sick_images,main_path)

In [ ]:
# Define constants
#IMAGE_SIZE = 224
#BATCH_SIZE = 32
TRAIN_SIZE = 0.7
VAL_SIZE = 0.15
TEST_SIZE = 0.15

## Data Augmentation

In [ ]:
# Define image size and batch size
img_size = (224, 224)
#batch_size = 16
#EPOCHS = 10

In [ ]:
# Define data augmentation parameters for the training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [ ]:
# Load the training set
train_set = train_datagen.flow_from_directory('E:/python/deep_learning_project/dataset1/datasets/training',
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='binary')

In [ ]:
# Load the test set
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory('E:/python/deep_learning_project/dataset1/datasets/testing',
                                                                   #target_size=img_size,
                                                                   target_size=(224, 224),
                                                                   batch_size=32,
                                                                   class_mode='binary',
                                                                   shuffle=False)

In [ ]:
# Load the validation set
val_set = ImageDataGenerator(rescale=1./255).flow_from_directory('E:/python/deep_learning_project/dataset1/datasets/validation',
                                                                  target_size=(224, 224),
                                                                  batch_size=32,
                                                                  class_mode='binary')

In [ ]:
# Define the CNN architecture
model = keras.models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
Dropout(0.5),
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model and setting callbacks for our model
early_stopping_monitor = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=5,
)

best_model = ModelCheckpoint('bestmodel.hdf5', monitor='val_accuracy', save_best_only=True)

history = model.fit(train_set,
           epochs= 10,
           batch_size=32,
           validation_data=val_set,)

In [ ]:
# Evaluate the model on the test data
prediction = model.predict(test_set)
y_pred = (prediction > 0.5).astype(int)

In [ ]:
# Evaluate the model on the test set
test_loss, accuracy = model.evaluate(test_set)
test_loss, precision = model.evaluate(test_set)
test_loss, recall = model.evaluate(test_set)
test_loss, f1score = model.evaluate(test_set)
print('Accuracy        :',(accuracy))
print('Precision       :',(precision))
print('Recall          :',(recall))
print('F1-score        :',(f1score))

In [ ]:
def generate_classification_report(pred, test_set):
    # Generate status classification report
    print(classification_report(pred, test_set.classes))

    # confusion matrix
    cm_status = confusion_matrix(test_set.classes, pred)

    # Plot status confusion matrix as heatmap
    sns.heatmap(cm_status, annot=True)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show

generate_classification_report(y_pred, test_set)

## Predicting the unknown images

In [ ]:
# Define paths to data and load into a list
unknown_images_path = 'E:/python/deep_learning_project/dataset1/unknown_images'
images = os.listdir(unknown_images_path)
unknown_images = []
for img_sets in images:
    unknown_image_path = f'{unknown_images_path}/{img_sets}'
    unknown_image = tf.keras.preprocessing.image.load_img(unknown_image_path, target_size=(224, 224))
    unknown_image = tf.keras.preprocessing.image.img_to_array(unknown_image)
    unknown_image = np.expand_dims(unknown_image, axis=0)
    unknown_images.append(unknown_image)

# Concatenate the unknown images into an array
unknown_images = np.concatenate(unknown_images, axis=0)

# Make predictions on the unknown images
predictions = model.predict(unknown_images)
predictions = (predictions > 0.5).astype(int)

results = []
result_labels = []

# Print the predictions
for i, prediction in enumerate(predictions):
    result_labels.append(f'Image {i+1}')
    results.append(prediction[0])
    
classes = np.argmax(predictions, axis = 1)
print(classes)

#create results dataframe
results_df = pd.DataFrame({'Image': result_labels, 'Prediction': results})

# save the dataframe to a CSV file
#results_df.to_csv('predictions.csv', index=False)

In [ ]:
# convert array into dataframe
DF = pd.DataFrame(predictions)
  
# save the dataframe as a csv file
DF.to_csv("image_results.csv")

In [ ]:
# Evaluate the model on the test set
test_loss, accuracy = model.evaluate(test_set)
test_loss, precision = model.evaluate(test_set)
test_loss, recall = model.evaluate(test_set)
test_loss, f1score = model.evaluate(test_set)
print('Accuracy        :',(accuracy))
print('Precision       :',(precision))
print('Recall          :',(recall))
print('F1-score        :',(f1score))

In [ ]:
#def process_image(img_sets):
    # Load an image
   # img = cv2.imread(img_sets)
   # img_filtered = cv2.GaussianBlur(img_sets, (3, 3), 0.5)
    #return results_df

In [ ]:
# Preditions on dataset 2

In [ ]:
dataset2_path = 'E:/python/deep_learning_project/dataset2'
test_set2 = tf.keras.utils.image_dataset_from_directory(dataset2_path,  shuffle=True,
  color_mode='rgb', image_size=(224, 224), batch_size=32)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

test_set2 = test_set2.cache().prefetch(buffer_size=AUTOTUNE)
#valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_set2 = test_set2.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_set2))
first_image = image_batch[3]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

In [ ]:
y_pred = []
y_true = []

for image_batch, label_batch in test_set2:

  y_true.append(label_batch)

  predictions = model.predict(image_batch)

  y_pred.append(np.argmax(predictions, axis = -1))

correct_labels = tf.concat([item for item in y_true],axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis =0)


test2_cm=confusion_matrix(predicted_labels,correct_labels)
print(test2_cm)

In [ ]:
# Retrieve a batch of images from the test set
image_batch, label_batch = test_set2.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch)

# Apply a sigmoid since our model returns logits
predictions_set2 = tf.math.argmax(predictions,1)
#predictions_ds2 = tf.where(predictions < 0.5, 0, 1)

print('Predictions Dataset2:\n', predictions_set2.numpy())
print('Labels:\n', label_batch)

#plt.figure(figsize=(20, 10))
#for i in range(32):
 # ax = plt.subplot(5, 10, i + 1)
 # plt.imshow(image_batch[i].astype("uint8"))

In [ ]:
# convert array into dataframe
DF = pd.DataFrame(predictions_set2)
  
# save the dataframe as a csv file
DF.to_csv("image_results_ds2.csv")

In [ ]:
test_loss,test_acc = model.evaluate(test_set2,verbose=2)
print(test_loss)
print(test_acc)

# Evaluate the model on the test set
test_loss, accuracy = model.evaluate(test_set2,verbose=2)
test_loss, precision = model.evaluate(test_set2,verbose=2)
test_loss, recall = model.evaluate(test_set2,verbose=2)
test_loss, f1score = model.evaluate(test_set2,verbose=2)
print('Accuracy        :',(accuracy))
print('Precision       :',(precision))
print('Recall          :',(recall))
print('F1-score        :',(f1score))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predicted_labels,correct_labels))